In [2]:
import speech_recognition as sr
import subprocess


In [3]:
r = sr.Recognizer()

In [4]:
harvard = sr.AudioFile('harvard.wav')
with harvard as source:
   audio = r.record(source)

In [4]:
r.recognize_google(audio)

'the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite a zestful food is be hot cross bun'

In [6]:
with harvard as source:
    audio1 = r.record(source, duration=4) # 0~4초
    audio2 = r.record(source, duration=4) # 4~8초

In [7]:
print(r.recognize_google(audio1))
print(r.recognize_google(audio2))

the stale smell of old beer lingers
it takes heat to bring out the odor a cold dip


In [5]:
with harvard as source:
    audio = r.record(source, offset=4, duration=3)  # 4~7초
    
r.recognize_google(audio)

'it takes heat to bring out the odor'

In [8]:
mic = sr.Microphone()

AttributeError: Could not find PyAudio; check installation

In [48]:
# with mic as source:
#     audio = r.listen(source)
    
# r.recognize_google(audio,language='ko-KR')

In [7]:
with mic as source:
    r.adjust_for_ambient_noise(source)
    audio = r.listen(source)
# raw = r.recognize_google(audio,language='ko-KR',show_all=True)
raw = r.recognize_google(audio,language='ko-KR')# 이름에 대한 인식..

# if raw in WORDS:
#     a = WORDS[0]
# a


NameError: name 'mic' is not defined

In [46]:
raw

'김준표'

In [44]:
raw.keys()

dict_keys(['alternative', 'final'])

In [11]:
# 학생 이름에 대한 데이터셋을 만들고 이름에 대한 인스턴스 불명확해도 기존 데이터셋에서 호출할 수 있도록 코드 작성
WORDS1 = ['화상 집','하당 집','화상집','가상 집']
WORDS2 = ['김준표','김중표']


In [47]:
if raw in WORDS2:
    print('김준표')

김준표


In [40]:
a = '김준표'
a
if a in WORDS2:
    print('김준표')

김준표


In [46]:
# with harvard as source:
#     audio = r.record(source, offset=4, duration=1)  # 4~7초
# if r.recognize_google(audio) == 'it takes':
#     subprocess.run('notepad')


In [40]:
# with mic as source:
#     r.adjust_for_ambient_noise(source)
#     audio = r.listen(source)    
    
# if r.recognize_google(audio) == 'hello':
#     subprocess.run('notepad')

In [51]:
import random
import time

import speech_recognition as sr


def recognize_speech_from_mic(recognizer, microphone):
    """Transcribe speech from recorded from `microphone`.

    Returns a dictionary with three keys:
    "success": a boolean indicating whether or not the API request was
               successful
    "error":   `None` if no error occured, otherwise a string containing
               an error message if the API could not be reached or
               speech was unrecognizable
    "transcription": `None` if speech could not be transcribed,
               otherwise a string containing the transcribed text
    """
    # check that recognizer and microphone arguments are appropriate type
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

    # adjust the recognizer sensitivity to ambient noise and record audio
    # from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    # set up the response object
    response = {
        "success": True,
        "error": None,
        "transcription": None
    }

    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught,
    #     update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


if __name__ == "__main__":
    # set the list of words, maxnumber of guesses, and prompt limit
    WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
    NUM_GUESSES = 3
    PROMPT_LIMIT = 5

    # create recognizer and mic instances
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    # get a random word from the list
    word = random.choice(WORDS)

    # format the instructions string
    instructions = (
        "I'm thinking of one of these words:\n"
        "{words}\n"
        "You have {n} tries to guess which one.\n"
    ).format(words=', '.join(WORDS), n=NUM_GUESSES)

    # show instructions and wait 3 seconds before starting the game
    print(instructions)
    time.sleep(3)

    for i in range(NUM_GUESSES):
        # get the guess from the user
        # if a transcription is returned, break out of the loop and
        #     continue
        # if no transcription returned and API request failed, break
        #     loop and continue
        # if API request succeeded but no transcription was returned,
        #     re-prompt the user to say their guess again. Do this up
        #     to PROMPT_LIMIT times
        for j in range(PROMPT_LIMIT):
            print('Guess {}. Speak!'.format(i+1))
            guess = recognize_speech_from_mic(recognizer, microphone)
            if guess["transcription"]:
                break
            if not guess["success"]:
                break
            print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

        # determine if guess is correct and if any attempts remain
        guess_is_correct = guess["transcription"].lower() == word.lower()
        user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
        if guess_is_correct:
            print("Correct! You win!".format(word))
            break
        elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
        else:
            print("Sorry, you lose!\nI was thinking of '{}'.".format(word))
            break


I'm thinking of one of these words:
apple, banana, grape, orange, mango, lemon
You have 3 tries to guess which one.

Guess 1. Speak!
I didn't catch that. What did you say?

Guess 1. Speak!
You said: Orange
Incorrect. Try again.

Guess 2. Speak!
I didn't catch that. What did you say?

Guess 2. Speak!
You said: mango
Incorrect. Try again.

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
I didn't catch that. What did you say?

Guess 3. Speak!
I didn't catch that. What did you say?

ERROR: Unable to recognize speech
